Excercise 1 : This excecise comes under 'Hands on Machine Learning' book where I have to create a decision tree regressor for 'Moons' datset


In [65]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

Fetching Data

In [66]:
X, y = make_moons(n_samples=10000, noise=0.4,random_state=42)
X.shape

(10000, 2)

Generating Random State

In [67]:
X = X.reshape((8000,2000))

ValueError: cannot reshape array of size 20000 into shape (8000,2000)

In [ ]:
train_X, test_X,train_y, test_y = train_test_split(X,y,test_size=0.2,random_state=42)


Now, Use Grid Search CV with cross-validation to fine tune the hyperparameters for Decision Tree Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=3)

grid_search_cv.fit(train_X, train_y)

grid_search_cv.best_estimator_

Now, Training it on full training set using these hyperparameters. and we are expected to get around 85% to 87% accuracy.

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = grid_search_cv.predict(test_X)
accuracy_score(test_y, y_pred)

Continuing Excercise : Generate 1000 subsets of the training set, each containing 100 instances selected randomly. 

In [ ]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []
ss = ShuffleSplit(n_splits=n_trees, test_size=len(train_X)-n_instances, random_state=42)

for mini_train_index,  mini_test_index in ss.split(train_X):
    X_mini_train = train_X[mini_train_index]
    y_mini_train = train_y[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

b.  Train one Decision Tree on each subset , using the best hyperparameter values found above. Evaluate these 1000 Decison Trees on the test set. Since they were trained on smalller sets, these Decision Trees will likely perform worse than the first Decision tree, acheiving only about 80% accuracy.

In [73]:
from sklearn.base import clone

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(test_X)
    accuracy_scores.append(accuracy_score(test_y, y_pred))

np.mean(accuracy_scores)

    

0.8054499999999999

Now, for each test set instance, generate the predictions of the 1000 Decision Trees, and keep only the most frequent prediction . this gives majority vote predictions over the test set.

In [75]:
Y_pred = np.empty([n_trees, len(test_X)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(test_X)
    

In [76]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

d. Evaluate these predictions on the test set. You should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher).

In [81]:
accuracy_score(test_y, y_pred_majority_votes.reshape([-1]))

0.872